In [1]:
import pandas as pd

In [2]:
INPUT_FILE = "./data/processed/freqs_dict.csv"

VERB = "VERB"
NOUN = "NOUN"
ADJ = "ADJ"
ADV = "ADV"
OUTPUTFILE = "output_file"
TOP_PERCENTILE = "top_percentile"
POSES = {
    VERB: {
        OUTPUTFILE : "./data/reports/top_verbs.csv",
        TOP_PERCENTILE : 98,
    },
    NOUN: {
        OUTPUTFILE : "./data/reports/top_nouns.csv",
        TOP_PERCENTILE : 99.5,
    },
    ADJ: {
        OUTPUTFILE : "./data/reports/top_adjectives.csv",
        TOP_PERCENTILE : 99,
    },
    ADV: {
        OUTPUTFILE : "./data/reports/top_adverbs.csv",
        TOP_PERCENTILE : 99,
    },
}

LEMMA = "lemma"
POS = "pos"
FREQUENCY = "frequency"

In [3]:
df = pd.read_csv(INPUT_FILE, index_col=None, header=None, skiprows=1, names=[LEMMA, POS, FREQUENCY])

# Exploration

In [4]:
print(f"Total of {df.shape[0]} instances containing {df[LEMMA].nunique()} distinct lemmas with {df[POS].nunique()} POS(s) were loaded.\n")

display(df.groupby(POS).count())

df.sample(10)

Total of 995789 instances containing 943965 distinct lemmas with 17 POS(s) were loaded.



,lemma,frequency
pos,,
ADJ,46542,46542
ADP,790,790
ADV,19319,19319
AUX,399,399
CCONJ,136,136
DET,740,741
INTJ,30,30
NOUN,412404,412407
NUM,314404,314406


,lemma,pos,frequency
974335,Modetrends,NOUN,1
598741,Colette-Einkäufer,NOUN,1
702251,6-Klässler,NOUN,1
620761,eingetauschen,VERB,1
335330,224508,NUM,3
780356,Klappspat,NOUN,1
142004,38439,NUM,3
927854,PostgreSQL-Community,NOUN,1
692305,Amtsenthebungsartikel,NOUN,1
366473,stichprobenartig,ADJ,3


## Missing values

In [5]:
df.isna().sum()

lemma        14
pos           0
frequency     0
dtype: int64

In [6]:
df[df.isna().any(axis="columns")]

,lemma,pos,frequency
5255,NaN,NUM,155
30288,NaN,NOUN,16
54021,NaN,PROPN,8
423085,NaN,PROPN,3
442342,NaN,PROPN,2
452809,NaN,PRON,2
473836,NaN,PROPN,2
490181,NaN,PROPN,2
566239,NaN,NOUN,1
617170,NaN,X,1


## Duplicate lemma-pos pairs

In [7]:
df[df.duplicated(subset=[LEMMA, POS], keep=False)].sort_values(by=[LEMMA, POS])

,lemma,pos,frequency
30288,NaN,NOUN,16
566239,NaN,NOUN,1
758022,NaN,NOUN,1
5255,NaN,NUM,155
643141,NaN,NUM,1
54021,NaN,PROPN,8
423085,NaN,PROPN,3
442342,NaN,PROPN,2
473836,NaN,PROPN,2
490181,NaN,PROPN,2


# Processing

## Missing values and duplicates

In [8]:
# Rmoving missing values will take care of duplicate lemma-pos pairs as well
df = df.dropna()
print(f"Total missing values after imputation: {df.isna().sum().sum()}")
print(f"Total duplicated lemma-pos pairs: {df[df.duplicated(subset=[LEMMA, POS])].shape[0]}")

Total missing values after imputation: 0
Total duplicated lemma-pos pairs: 0


## Invalid lemmas

In [9]:
# Removing all-caps lemmas
df = df[~df[LEMMA].str.isupper()]

# Keeping ONLY alphbetic while accepting hyphens
df = df[df[LEMMA].str.match(r'^(?=.*[A-Za-zÄÖÜäöüß])[A-Za-zÄÖÜäöüß]+(-[A-Za-zÄÖÜäöüß]+)*$')]

# Removing the leading and trailing hyphens
df[LEMMA] = df[LEMMA].str.strip("-")

df

,lemma,pos,frequency
0,der,DET,1128730
1,in,ADP,307254
2,und,CCONJ,286005
3,ein,DET,269419
4,sein,AUX,227051
...,...,...,...
995775,CIK-Zelle,NOUN,1
995776,mononukleär,ADJ,1
995777,Zzt,NOUN,1
995778,Beuth,ADJ,1


## Cleaning up POSes

In [10]:
all_poses = df.groupby(by=[POS])[LEMMA].count()
all_poses

pos
ADJ       43598
ADP         489
ADV       17784
AUX         302
CCONJ        93
DET         544
INTJ         28
NOUN     390814
NUM         313
PART         39
PRON        599
PROPN    132764
PUNCT       110
SCONJ       105
SPACE         1
VERB      21640
X         12628
Name: lemma, dtype: int64

In [11]:
df = df.drop(df[~df[POS].isin(POSES.keys())].index)

## Cleaning up lemmas

In [12]:
df_verb = df[df[POS] == VERB]

df_verb.sort_values(FREQUENCY, ascending=False)

,lemma,pos,frequency
34,geben,VERB,25024
47,kommen,VERB,16581
53,finden,VERB,15318
57,gehen,VERB,13723
60,stehen,VERB,13375
...,...,...,...
995474,herantragen,VERB,1
995616,runtergenudeln,VERB,1
995648,vagabundieren,VERB,1
995670,vertrags,VERB,1


In [13]:
# Verb lemmas that do not end in "n"
df_verb_lemmas_not_ending_in_n = df_verb[
    ~((df_verb[LEMMA].str.endswith("n")) | (df_verb[LEMMA].str.endswith("N")))]


df_verb_lemmas_not_ending_in_n.sort_values(FREQUENCY, ascending=False)

,lemma,pos,frequency
332,lässt,VERB,2808
1383,sodass,VERB,774
1644,kannst,VERB,656
1663,schloss,VERB,646
3139,zusammengefasst,VERB,307
...,...,...,...
994684,Las,VERB,1
995078,Literaturpädagogik,VERB,1
995092,Gebraucht,VERB,1
995281,Tolerantia,VERB,1


In [14]:
df = df.drop(df_verb_lemmas_not_ending_in_n.index)

In [15]:
# Remove the duplicate POS-LEMMA pairs
not_noun_filter = df[POS] != NOUN
df.loc[not_noun_filter, LEMMA] = df.loc[not_noun_filter, LEMMA].str.lower()

df[FREQUENCY] = df.groupby([POS, LEMMA])[FREQUENCY].transform("sum")

max_freq_indexes = df.groupby([POS, LEMMA])[FREQUENCY].idxmax()
df = df.loc[max_freq_indexes].sort_values(by=[POS, LEMMA], ascending=False).reset_index(drop=True)

df

,lemma,pos,frequency
0,ühren,VERB,1
1,übrigbleiben,VERB,13
2,überzusiedeln,VERB,1
3,überzulaufen,VERB,1
4,überzugehen,VERB,10
...,...,...,...
463608,aalglatt,ADJ,3
463609,aalener,ADJ,5
463610,aalartig,ADJ,2
463611,aachenmünchener,ADJ,1


## Filtering the most frequents

In [16]:
df.groupby(POS)[LEMMA].count()

pos
ADJ      41720
ADV      16173
NOUN    390814
VERB     14906
Name: lemma, dtype: int64

In [17]:
def select_top_percentile(pos: str, percentile: float) -> pd.DataFrame:
    df_pos = df[df[POS] == pos]
    top_quantile = df_pos[FREQUENCY].quantile(percentile/100)
    return df_pos[df_pos[FREQUENCY] >= top_quantile].sort_values(FREQUENCY, ascending=False)

output_dfs = dict()
for pos in POSES.keys():
    output_dfs[pos] = select_top_percentile(pos, POSES[pos][TOP_PERCENTILE])

for key,value in output_dfs.items():
    print(f"{key} -> {len(value)}")

VERB -> 299
NOUN -> 1955
ADJ -> 418
ADV -> 163


# Exporting the results

In [18]:
for pos, output_df in output_dfs.items():
    output_df[LEMMA].to_csv(POSES[pos][OUTPUTFILE], index=None, header=False)